#### Google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
path = '/content/drive/MyDrive/final_project_itc/data.npy'
data = np.load(path, allow_pickle = True)

In [4]:
df = pd.DataFrame(data, columns = ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'])

In [5]:
df = df.loc[:,['ProductId','Text','Score']]

In [6]:
! pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=cdca70dd4122eb8ed4e8da1e5bd6e5f1aaa1fa38d78b70510ea8e58c6375e178
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [7]:
from transformers import pipeline
classifier = pipeline(task="zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
df.shape

(568454, 3)

In [9]:
all_reviews = df.Text.iloc[:100].to_list()

In [10]:
candidate_labels = ['flavor','taste','texture', 'price','expiration date','quality', 'size','delivery','packaging']

In [22]:
hypothesis_template = "The topic of this review is {}."

In [29]:
single_topic_prediction = classifier(all_reviews, candidate_labels, hypothesis_template=hypothesis_template)

In [24]:
single_topic_prediction = pd.DataFrame(single_topic_prediction)

In [25]:
single_topic_prediction['predicted_topic'] = single_topic_prediction['labels'].apply(lambda x: x[0])
single_topic_prediction['predicted_topic_score'] = single_topic_prediction['scores'].apply(lambda x: x[0])
single_topic_prediction.head()

,sequence,labels,scores,predicted_topic,predicted_topic_score
0,I have bought several of the Vitality canned d...,"[quality, flavor, taste, texture, packaging, s...","[0.5272221565246582, 0.17711590230464935, 0.11...",quality,0.527222
1,Product arrived labeled as Jumbo Salted Peanut...,"[size, packaging, delivery, taste, quality, fl...","[0.2656744420528412, 0.24203531444072723, 0.16...",size,0.265674
2,This is a confection that has been around a fe...,"[flavor, taste, texture, quality, delivery, si...","[0.36638304591178894, 0.2520144283771515, 0.16...",flavor,0.366383
3,If you are looking for the secret ingredient i...,"[flavor, taste, quality, delivery, packaging, ...","[0.6471533179283142, 0.22996950149536133, 0.06...",flavor,0.647153
4,Great taffy at a great price. There was a wid...,"[price, delivery, taste, flavor, quality, pack...","[0.5207945704460144, 0.36563047766685486, 0.04...",price,0.520795


In [28]:
single_topic_prediction['sequence'].iloc[2]

'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.'

In [15]:
single_topic_prediction.predicted_topic.value_counts()

taste        27
quality      25
flavor       23
price        12
delivery      5
texture       4
packaging     3
size          1
Name: predicted_topic, dtype: int64

In [16]:
single_topic_prediction['sequence'].iloc[4]

'Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.'

In [17]:
np.where(single_topic_prediction.predicted_topic == 'price')

(array([ 4, 19, 27, 36, 37, 60, 61, 68, 87, 90, 96, 99]),)

In [18]:
multi_topic_prediction = classifier(all_reviews, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True)

multi_topic_prediction = pd.DataFrame(multi_topic_prediction)


multi_topic_prediction.head()

,sequence,labels,scores
0,I have bought several of the Vitality canned d...,"[quality, flavor, texture, taste, packaging, d...","[0.9933328628540039, 0.7305848002433777, 0.211..."
1,Product arrived labeled as Jumbo Salted Peanut...,"[packaging, size, delivery, quality, flavor, t...","[0.9935596585273743, 0.9891250133514404, 0.936..."
2,This is a confection that has been around a fe...,"[taste, flavor, quality, texture, delivery, si...","[0.943139374256134, 0.9410188794136047, 0.4016..."
3,If you are looking for the secret ingredient i...,"[flavor, taste, quality, delivery, texture, pa...","[0.9709861874580383, 0.5440918803215027, 0.053..."
4,Great taffy at a great price. There was a wid...,"[price, delivery, flavor, taste, quality, text...","[0.9931796789169312, 0.9688953757286072, 0.455..."


In [19]:
multi_topic_prediction.sequence.iloc[4]

'Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.'

In [20]:
threshold = 0.6

multi_topic_prediction = multi_topic_prediction.set_index('sequence').apply(pd.Series.explode).reset_index()
multi_topic_prediction = multi_topic_prediction[multi_topic_prediction['scores'] >= threshold]
multi_topic_prediction.head()

,sequence,labels,scores
0,I have bought several of the Vitality canned d...,quality,0.993333
1,I have bought several of the Vitality canned d...,flavor,0.730585
9,Product arrived labeled as Jumbo Salted Peanut...,packaging,0.99356
10,Product arrived labeled as Jumbo Salted Peanut...,size,0.989125
11,Product arrived labeled as Jumbo Salted Peanut...,delivery,0.936342


In [21]:
multi_topic_prediction.labels.value_counts()

taste        56
quality      55
flavor       52
price        17
delivery     13
texture      10
packaging     8
size          7
Name: labels, dtype: int64